In [57]:
# We will do the imports required for this notebook here
%load_ext autoreload
%autoreload 2
# numpy provides import array and linear algebra utilities
import numpy as np

# the robotics toolbox provides robotics specific functionality
import roboticstoolbox as rtb

# spatial math provides objects for representing transformations
import spatialmath as sm

# swift is a lightweight browser-based simulator which comes eith the toolbox
from swift import Swift

# spatialgeometry is a utility package for dealing with geometric objects
import spatialgeometry as sg

# import urchin to read urdf files
from urchin import URDF

# this package provides several solvers for solving quadratic programmes
import qpsolvers as qp

# modern robotics code library accompanying 
# Modern Robotics: Mechanics, Planning, and Control (Kevin Lynch and Frank Park)
import modern_robotics as mr

# the os package provides utilities for interacting with the operating system
import os

# import config 
from config.config import *
from models.panda_model import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# Make the environment
env = Swift()

# Launch the simulator
# The realtime flag will ask the simulator to simulate as close as
# possible to realtime as apposed to as fast as possible
env.launch(realtime=True, browser="notebook")


panda_virtual = create_virtual_panda()
# We can then add our robot to the simulator envionment
env.add(panda_virtual)

# end-effector axes
ee_axes = sg.Axes(0.1)
ee_axes.T = panda_virtual.fkine(panda_virtual.q, end='panda_finger_virtual')

# goal axes
goal_axes = sg.Axes(0.1)

# Add the axes to the environment
env.add(ee_axes)
env.add(goal_axes) 

# Make a variable for the upper and lower limits of the robot
qd_lb = -20.0 * np.ones(panda_virtual.n)
qd_ub = 20.0 * np.ones(panda_virtual.n)

In [61]:
env = Swift()
# Launch the simulator
# The realtime flag will ask the simulator to simulate as close as
# possible to realtime as apposed to as fast as possible
env.launch(realtime=True, browser="notebook")
panda = rtb.models.Panda()
frankie = rtb.models.Frankie()
box = sg.Cuboid(scale=[0.1, 0.07, 0.03],
                color='blue')
box.T = sm.SE3(0.7, 0, 0.015)
box.set_alpha(0.5)

panda_virtual.q = panda_virtual.qr
# open the fingers
panda_virtual.grippers[0].q = [0.035, 0.035]
# Set a desired and effector pose an an offset from the current end-effector pose
bTe = sm.SE3(-box.scale[0]/2 + 0.01, 0, 0) * sm.SE3.Rx(np.pi/2)
Tep = box.T * bTe * sm.SE3.Rz(0)

# Set the goal axes to Tep
goal_axes.T = Tep
# Add the robot to the simulator
env.add(panda_virtual)
env.add(box)



# Simulate the robot while it has not arrived at the goal
arrived = False
while not arrived:

    # Work out the required end-effector velocity to go towards the goal
    Te = panda_virtual.fkine(panda_virtual.q, end='panda_finger_virtual')
    v, arrived = rtb.p_servo(Te, Tep, threshold=0.01)
    
    # Set the Panda's joint velocities
    panda_virtual.qd = np.linalg.pinv(panda_virtual.jacobe(panda_virtual.q, end='panda_finger_virtual')) @ v
    
    # Step the simulator by 50 milliseconds
    env.step(0.05)


In [81]:
Je = panda_virtual.jacobe(panda_virtual.qz, end='panda_finger_virtual')
J0 = panda_virtual.jacob0(panda_virtual.qz, end='panda_finger_virtual')


print(np.round(Je, 3))
print(np.round(J0, 3))
print(panda_virtual.qz)

[[ 0.062  0.346  0.062 -0.123  0.062  0.149 -0.     0.   ]
 [ 0.    -0.088  0.     0.005  0.     0.088  0.     0.   ]
 [-0.062  0.346 -0.062 -0.123 -0.062  0.149  0.     0.   ]
 [-0.     0.707 -0.    -0.707 -0.    -0.707  0.     0.   ]
 [ 1.     0.     1.    -0.     1.    -0.    -1.     0.   ]
 [ 0.    -0.707  0.     0.707  0.     0.707  0.     1.   ]]
[[ 0.     0.49   0.    -0.174  0.     0.21   0.     0.   ]
 [ 0.088  0.     0.088  0.     0.088  0.     0.     0.   ]
 [ 0.    -0.088 -0.     0.005 -0.     0.088  0.     0.   ]
 [-0.     0.     0.     0.     0.     0.     0.     0.707]
 [ 0.     1.     0.    -1.     0.    -1.    -0.    -0.707]
 [ 1.     0.     1.     0.     1.     0.    -1.     0.   ]]
[0. 0. 0. 0. 0. 0. 0. 0.]


In [48]:
from utils.ik_jl import *

ets = panda_virtual.ets(end='panda_finger_virtual')
iksolver = IK_JL(ets, k0=[1]*7 + [0])

iksol = iksolver.solve(ets=ets, Tep=Tep)

In [ ]:
def isarrived(Te, Tep):
    return 

In [47]:
def secondary_object(robot: rtb.Robot, q: np.ndarray) -> np.ndarray:
    """
    Calculate the gradient of the joint limit avoidance function

    Parameters
    ----------
    robot
        The robot the manipulators kinematics
    q
        The current joint coordinate vector

    Returns
    -------
    dw_dq
        The gradient of the joint limit avoidance function
    """
    n = robot.n
    qlim = robot.qlim
    
    q_mid = (qlim[1] + qlim[0]) / 2
    q_range = qlim[1] - qlim[0]
    
    # Vectorized calculation with singularity protection
    with np.errstate(divide='ignore', invalid='ignore'):
        dw_dq = -1/n * (q - q_mid) / (q_range)**2
        
    # Handle unlimited joints (range = 0)
    dw_dq[np.isinf(dw_dq)] = 0  
    dw_dq[np.isnan(dw_dq)] = 0
    
    return dw_dq

array([4.2432671 , 6.07493002, 2.64662446, 3.22677391, 1.18278114,
       3.60330011])

In [72]:

env = Swift()
# Launch the simulator
# The realtime flag will ask the simulator to simulate as close as
# possible to realtime as apposed to as fast as possible
env.launch(realtime=True, browser="notebook")
ee_axes = sg.Axes(0.1)
env.add(panda_virtual)
env.add(ee_axes)
env.add(box)


arrived = False
# Specify the gain for the p_servo method
kt = 5.0
kr = 2.0
k = np.array([kt, kt, kt, kr, kr, kr])
n = panda_virtual.n
k0 = 0.1
Vep = np.empty(6)
dt = 0.05
panda_virtual.q = panda_virtual.qr
# Run the simulation until the robot arrives at the goal
while not arrived:

    # Work out the base frame manipulator Jacobian using the current robot configuration
    J = panda_virtual.jacobe(panda_virtual.q, end='panda_finger_virtual')
    J_pinv = np.linalg.pinv(J)

    # The end-effector pose of the panda (using .A to get a numpy array instead of an SE3 object)
    Te = panda_virtual.fkine(panda_virtual.q, end='panda_finger_virtual').A

    # Calculate the required end-effector velocity and whether the robot has arrived
    ev, arrived = rtb.p_servo(Te, Tep, gain=k, threshold=0.01, method="screw")

    dw_dq = secondary_object(panda_virtual, panda_virtual.q)
    q0 = k0 * dw_dq
    qd = J_pinv @ (Vep + ev) + (np.eye(n) - J_pinv @ J) @ q0
    print(ev)


    # Apply the joint velocities to the Panda
    panda_virtual.qd[:panda_virtual.n] = qd[:panda_virtual.n]

    # Update the ee axes
    ee_axes.T = Te

    # Step the simulator by dt seconds
    env.step(dt)

[ 7.79625219e-01 -2.03048011e+00 -3.17568930e-11  9.15541338e-12
  6.23312254e-12 -2.00000000e-01]
[ 0.06428492 -2.52350049 -2.25521564  0.78288597 -0.03039159 -0.18007486]
[-0.75798301 -2.85817487 -2.5363828   0.93003904 -0.73813158 -0.73585624]
[ 2.19426146 -0.82699532 -2.16859893  4.54968761  1.20683822  0.90621787]
[ 2.06974348 -0.06456566 -3.95533109  3.51385854  1.76427331  1.36210107]
[ 0.89008077 -2.88814776 -1.232381    2.06862142  0.41895281  0.78147961]
[-0.33204252 -4.16591209 -1.36673003  0.89103179  0.29547668 -0.02947048]
[-1.29683874 -1.10281295  0.75479667 -0.95940071 -2.39774686  2.09610527]
[-0.44029821 -1.6751659   6.68022865  0.31959689 -5.58860836  0.57964157]
[-2.85843436 -3.48644602  4.66725076  0.84967127 -5.06133264  0.74252203]
[-3.1567099   0.18501298  3.53353113 -1.26042146 -3.96337845  2.05081291]
[ 0.95727111 -2.25348136  5.18887288 -2.3125508  -3.98156602 -0.23614578]
[-0.13813226 -1.09484885  1.31905726 -1.98199341 -3.55132737 -0.31014753]
[-2.31875741 

connection handler failed
Traceback (most recent call last):
  File "/home/vahdan/miniconda3/envs/panda/lib/python3.10/site-packages/websockets/legacy/server.py", line 245, in handler
    await self.ws_handler(self)
  File "/home/vahdan/miniconda3/envs/panda/lib/python3.10/site-packages/websockets/legacy/server.py", line 1190, in _ws_handler
    return await cast(
  File "/home/vahdan/repos/IJRR/swift/swift/SwiftRoute.py", line 320, in serve
    await self.expect_message(websocket, expected)
  File "/home/vahdan/repos/IJRR/swift/swift/SwiftRoute.py", line 325, in expect_message
    recieved = await websocket.recv()
  File "/home/vahdan/miniconda3/envs/panda/lib/python3.10/site-packages/websockets/legacy/protocol.py", line 562, in recv
    await self.ensure_open()
  File "/home/vahdan/miniconda3/envs/panda/lib/python3.10/site-packages/websockets/legacy/protocol.py", line 929, in ensure_open
    raise self.connection_closed_exc()
websockets.exceptions.ConnectionClosedOK: received 1001 (g

[ 5.8571061  -5.48894781 -1.28296178 -1.90036368  1.50303376 -2.0408579 ]
[ 5.8571061  -5.48894782 -1.28296177 -1.90036365  1.50303375 -2.04085791]


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# import genesis as gs

# gs.init(backend=gs.cuda)

# scene = gs.Scene(
#     show_viewer = True,
#     viewer_options = gs.options.ViewerOptions(
#         res           = (1280, 960),
#         camera_pos    = (3.5, 0.0, 2.5),
#         camera_lookat = (0.0, 0.0, 0.5),
#         camera_fov    = 40,
#         max_FPS       = 60,
#     ),
#     vis_options = gs.options.VisOptions(
#         show_world_frame = True,
#         world_frame_size = 1.0,
#         show_link_frame  = False,
#         show_cameras     = False,
#         plane_reflection = True,
#         ambient_light    = (0.1, 0.1, 0.1),
#     ),
#     renderer=gs.renderers.Rasterizer(),
# )

# plane = scene.add_entity(
#     gs.morphs.Plane(),
# )
# franka = scene.add_entity(
#     gs.morphs.MJCF(file='xml/franka_emika_panda/panda.xml'),
# )

# cam = scene.add_camera(
#     res    = (640, 480),
#     pos    = (3.5, 0.0, 2.5),
#     lookat = (0, 0, 0.5),
#     fov    = 30,
#     GUI    = False,
# )

# scene.build()

# # render rgb, depth, segmentation, and normal
# # rgb, depth, segmentation, normal = cam.render(rgb=True, depth=True, segmentation=True, normal=True)

# cam.start_recording()
# import numpy as np

# for i in range(120):
#     scene.step()
#     cam.set_pose(
#         pos    = (3.0 * np.sin(i / 60), 3.0 * np.cos(i / 60), 2.5),
#         lookat = (0, 0, 0.5),
#     )
#     cam.render()
# cam.stop_recording(save_to_filename='video.mp4', fps=60)

In [ ]:
# import numpy as np
# import genesis as gs

# ########################## init ##########################
# gs.init(backend=gs.cuda)

# ########################## create a scene ##########################
# scene = gs.Scene(
#     show_viewer = True,
#     viewer_options = gs.options.ViewerOptions(
#         res           = (1280, 960),
#         camera_pos    = (3.5, 0.0, 2.5),
#         camera_lookat = (0.0, 0.0, 0.5),
#         camera_fov    = 40,
#         max_FPS       = 60,
#     ),
#     vis_options = gs.options.VisOptions(
#         show_world_frame = True,
#         world_frame_size = 1.0,
#         show_link_frame  = False,
#         show_cameras     = False,
#         plane_reflection = True,
#         ambient_light    = (0.1, 0.1, 0.1),
#     ),
#     renderer=gs.renderers.Rasterizer(),
# )

# cam = scene.add_camera(
#     res    = (640, 480),
#     pos    = (3.5, 0.0, 2.5),
#     lookat = (0, 0, 0.5),
#     fov    = 30,
#     GUI    = False,
# )


# ########################## entities ##########################
# plane = scene.add_entity(
#     gs.morphs.Plane(),
# )
# cube = scene.add_entity(
#     gs.morphs.Box(
#         size = (0.04, 0.04, 0.04),
#         pos  = (0.65, 0.0, 0.02),
#     )
# )
# franka = scene.add_entity(
#     gs.morphs.MJCF(file='xml/franka_emika_panda/panda.xml'),
# )
# ########################## build ##########################
# scene.build()

# motors_dof = np.arange(7)
# fingers_dof = np.arange(7, 9)

# # set control gains
# # Note: the following values are tuned for achieving best behavior with Franka
# # Typically, each new robot would have a different set of parameters.
# # Sometimes high-quality URDF or XML file would also provide this and will be parsed.
# franka.set_dofs_kp(
#     np.array([4500, 4500, 3500, 3500, 2000, 2000, 2000, 100, 100]),
# )
# franka.set_dofs_kv(
#     np.array([450, 450, 350, 350, 200, 200, 200, 10, 10]),
# )
# franka.set_dofs_force_range(
#     np.array([-87, -87, -87, -87, -12, -12, -12, -100, -100]),
#     np.array([ 87,  87,  87,  87,  12,  12,  12,  100,  100]),
# )

In [ ]:

# # get the end-effector link
# end_effector = franka.get_link('hand')

# # move to pre-grasp pose
# qpos = franka.inverse_kinematics(
#     link = end_effector,
#     pos  = np.array([0.65, 0.0, 0.25]),
#     quat = np.array([0, 1, 0, 0]),
# )
# # gripper open pos
# qpos[-2:] = 0.04
# path = franka.plan_path(
#     qpos_goal     = qpos,
#     num_waypoints = 200, # 2s duration
# )
# # execute the planned path
# for waypoint in path:
#     franka.control_dofs_position(waypoint)
#     scene.step()

# # allow robot to reach the last waypoint
# for i in range(100):
#     scene.step()
# # reach
# qpos = franka.inverse_kinematics(
#     link = end_effector,
#     pos  = np.array([0.65, 0.0, 0.130]),
#     quat = np.array([0, 1, 0, 0]),
# )
# franka.control_dofs_position(qpos[:-2], motors_dof)
# for i in range(100):
#     scene.step()

# # grasp
# franka.control_dofs_position(qpos[:-2], motors_dof)
# franka.control_dofs_force(np.array([-0.5, -0.5]), fingers_dof)

# for i in range(100):
#     scene.step()

# # lift
# qpos = franka.inverse_kinematics(
#     link=end_effector,
#     pos=np.array([0.65, 0.0, 0.28]),
#     quat=np.array([0, 1, 0, 0]),
# )
# franka.control_dofs_position(qpos[:-2], motors_dof)
# for i in range(200):
#     scene.step()